In [1]:
import matplotlib.pyplot as plt
import plotly
import numpy as np
import random
import hrr
import math
import time
from plotly.graph_objs import Scatter, Layout, Surface
plotly.offline.init_notebook_mode(connected=True)

In [2]:
def log_transform(error):
    return math.copysign(1.0,error)*math.log(math.fabs(error)+1,2)

In [3]:
def softmax(arr,t=1.0):
    w = np.array(arr)
    e = np.exp(w / t)
    dist = e / np.sum(e)
    return dist

In [4]:
def argmax(arr_2d,restrict):
    max_row = restrict[0]
    max_col = 0
    #max_value = arr_2d[0,0]
    max_value = arr_2d[restrict[0],0]
    for row in range(arr_2d.shape[0]):
        if row not in restrict:
            continue
        for col in range(arr_2d.shape[1]):
            if arr_2d[row,col] > max_value:
                max_value = arr_2d[row,col]
                max_row,max_col = row,col
    return list((max_row,max_col))

In [5]:
def argmax1(arr_3d,outer,inner):
    max_row = outer[0]
    max_col = inner[0]
    max_x = 0
    #max_value = arr_2d[0,0]
    max_value = arr_3d[outer[0],inner[0],0]
    for row in range(arr_3d.shape[0]):
        if row not in outer:
            continue
        for col in range(arr_3d.shape[1]):
            if col not in inner:
                continue
            for x in range(arr_3d.shape[2]):
                if arr_3d[row,col,x] > max_value:
                    max_value = arr_3d[row,col,x]
                    max_row,max_col,max_x = row,col,x
    return list((max_row,max_col,max_x))

In [45]:
def Delayed_Saccade(ntrials,lrate,gamma,td_lambda,temp):
    n = 10000
    n_cross_locations = 5
    n_dot_locations = 5
    n_gaze_locations = 5
    n_dot_colors = 4
    n_cross_colors = 4
    #n_saccades = 5 # actions
    
    # working memory chuncks
    n_wm_crosshair_location = 5
    n_wm_dot_location = 5
    n_wm_agent_fixation = 5
    
    # identity vector
    hrr_i = np.zeros(n)
    hrr_i[0] = 1
    
    # saccade hrr
    agent_gaze_location = hrr.hrrs(n,n_gaze_locations)
    agent_gaze_location = np.row_stack((agent_gaze_location,hrr_i))
    
    # crosshair location hrr
    crosshair_location = hrr.hrrs(n,n_cross_locations)
    crosshair_location = np.row_stack((crosshair_location,hrr_i))
    
    # dot location hrr
    dot_location = hrr.hrrs(n,n_dot_locations)
    dot_location = np.row_stack((dot_location,hrr_i))
    
    # color hrr
    dot_color = hrr.hrrs(n,n_dot_colors)
    dot_color = np.row_stack((dot_color,hrr_i))
    
    # crosshair hrr
    crosshair_color = hrr.hrrs(n,n_cross_colors)
    crosshair_color = np.row_stack((crosshair_color,hrr_i))

    # agent fixation working memory hrr
    agent_fixation_wm = hrr.hrrs(n,n_wm_agent_fixation)
    agent_fixation_wm = np.row_stack((agent_fixation_wm,hrr_i))
    
    # dot location working memory hrr
    dot_location_wm = hrr.hrrs(n,n_wm_dot_location)
    dot_location_wm = np.row_stack((dot_location_wm,hrr_i))
    
    # crosshair location working memory hrr
    crosshair_location_wm = hrr.hrrs(n,n_wm_crosshair_location)
    crosshair_location_wm = np.row_stack((crosshair_location_wm,hrr_i))
    #print(dot_location[-1])
    #tick1 = time.time()
    #print('starting precompute ...')
    # precompute hrrs
    
    #tick2 = time.time()
    #print("time spent precomputing:",(tick2-tick1),"seconds")
    #print(representation.shape)
    # weight vector and bias
    W = hrr.hrr(n)
    bias = 1
    
    # epsilon for e-soft policy
    epsilon = .1
    
    # temperature for softmax
    t = temp
    
    # eligibility trace
    eligibility = np.zeros(n)
    center = 4
    nothing = 5
    correct_response_count = 0
    count = 0
    for trial in range(ntrials):
        if trial % 10000 == 0:
            epsilon = 0.01
        count += 1
        r = 0
        flag_count = 0 # used to determine if agent completed all stages correct
        eligibility = np.zeros(n)
        
        ######### nothing on the screen #############
        representation = hrr.convolve(agent_gaze_location,hrr.convolve(crosshair_location[-1],dot_location[-1]))
        
        values = np.dot(representation,W) + bias
        sm_prob = softmax(values,t)
        #print(sm_prob.shape)
        fixation = np.argmax(sm_prob)
        
        if random.random() < epsilon:
            fixation = random.randrange(n_gaze_locations)
        
        value = values[fixation]
        #error = (r+gamma*value) - pvalue
        #error = r - value
        #W += lrate*log_transform(error)*eligibility
        r = 0
        pvalue = value
        pfixation = fixation
        #eligibility = representation[fixation,:] + td_lambda*eligibility
        #e = hrr.convolve(agent_gaze_location[fixation],hrr.convolve(crosshair_location[-1],dot_location[-1]))
        eligibility = representation[fixation,:] + td_lambda*eligibility
        ###########################################
        
       
        ######## just the crosshair on the screen ###########
        representation = hrr.convolve(agent_gaze_location,hrr.convolve(crosshair_location[center],dot_location[-1]))
        
        #print(representation.shape)
        values = np.dot(representation,W) + bias
        #print(values)
        #print(values.shape)
        sm_prob = softmax(values,t)
        #print(sm_prob)
        # bootstraps agent towards looking at center crosshair
        
        fixation = center#np.argmax(sm_prob)
        
        #fixation = np.argmax(sm_prob)
        
        #print(wm_cross,wm_agent,action)
        #epsilon = .2
        if random.random() < epsilon:
            fixation = random.randrange(n_gaze_locations)
        
        value = values[fixation]
        error = (r+gamma*value) - pvalue
        W += lrate*log_transform(error)*eligibility
        
        pvalue = value
        pfixation = fixation
        
        eligibility = representation[fixation,:] + td_lambda*eligibility
        #e = hrr.convolve(agent_gaze_location[fixation],hrr.convolve(crosshair_location[-1],dot_location[-1]))
         
        if fixation == center:
            flag_count += 1
            r = 0
        
        #print(fixation,end=" ")
        ####################
        
        ########## crosshair and colored dot on the screen #######
        dot_loc = random.randint(0,3)
        representation = hrr.convolve(agent_gaze_location,hrr.convolve(crosshair_location[center],dot_location[dot_loc]))
        #wm = hrr.oconvolve(agent_fixation_wm,hrr.oconvolve(dot_location_wm,crosshair_location_wm))
        representation = hrr.oconvolve(representation,dot_location_wm)
        representation = representation.reshape((n_gaze_locations+1,n_wm_dot_location+1,n))
        #representation = representation.reshape((n_gaze_locations+1,n))
        #print(representation.shape)
        values = np.dot(representation,W) + bias
        sm_prob = softmax(values,t)
        
        possible_wm = np.unique(np.array([dot_loc]))
        wm_action = argmax(sm_prob,possible_wm)
        
        wm_dot = wm_action[0]
        # bootstraps agent towards looking at center crosshair
        
        fixation = center#np.argmax(sm_prob)
        
        #fixation = wm_action[1]
        #print('wm:',wm_dot,dot_loc)
        #epsilon = .2
        if random.random() < epsilon:
            fixation = random.randrange(n_gaze_locations)
            wm_dot = random.randrange(n_gaze_locations-1)
        value = values[wm_dot,fixation]
        #value = values[fixation]
        # update error and weights
        error = (r+gamma*value) - pvalue
        W += lrate*log_transform(error)*eligibility
        
        pvalue = value
        pfixation = fixation
        pwm = wm_dot
        
        eligibility = representation[fixation,wm_dot,:] + td_lambda*eligibility
        #eligibility = hrr.convolve(representation[fixation,:],dot_location_wm[wm_dot]) + td_lambda*eligibility
        if fixation == center:
            flag_count += 1
            r = 0
        
        #print(fixation,end=" ")
        ######################
        
        ####### just the crosshair on the screen ################
        
        representation = hrr.convolve(agent_gaze_location,hrr.convolve(crosshair_location[center],dot_location[-1]))
        representation = hrr.convolve(representation,dot_location_wm[wm_dot])
       
        representation = representation.reshape((n_gaze_locations+1,n))
        #print(representation.shape)
        values = np.dot(representation,W) + bias
        #print(values)
        #print(values.shape)
        sm_prob = softmax(values,t)
        #print(sm_prob)
        
        
        #print(wm_cross,wm_agent,action)
        #epsilon = .2
        if random.random() < epsilon:
            fixation = random.randrange(n_gaze_locations+1)
        
        value = values[fixation]
        error = (r+gamma*value) - pvalue
        W += lrate*log_transform(error)*eligibility
        
        pvalue = value
        pfixation = fixation
        pwm = wm_dot
        #eligibility = representation[fixation,wm_agent,wm_crosshair,wm_dot,:] + td_lambda*eligibility
        #e = hrr.convolve(agent_gaze_location[fixation],hrr.convolve(crosshair_location[-1],dot_location[-1]))
        eligibility = representation[fixation,:] + td_lambda*eligibility
        if fixation == center:
            flag_count += 1
            r = 0
        
        #print(fixation,end=" ")
        ##############################
        
        ######## remove crosshair on the screen ################
        representation = hrr.convolve(agent_gaze_location,hrr.convolve(crosshair_location[-1],dot_location[-1]))
        representation = hrr.convolve(representation,dot_location_wm[wm_dot])
        values = np.dot(representation,W) + bias
        sm_prob = softmax(values,t)
        #print(sm_prob.shape)
        fixation = np.argmax(sm_prob)
        
        # updated info
        
        if random.random() < epsilon:
            fixation = random.randrange(n_gaze_locations)
            
        
        if fixation == dot_loc:
            flag_count += 1
            r = 0
      
        value = values[fixation]
        error = (r+gamma*value) - pvalue
        #error = r - value
        W += lrate*log_transform(error)*eligibility
        
        pvalue = value
        pfixation = fixation
        eligibility = representation[fixation,:] + td_lambda*eligibility

        
        #############################
        
        #print('dot_loc:','(',dot_loc,')',end = ' ')
        #epsilon = .4
        '''
        if random.random() < epsilon:
            fixation = random.randrange(n_gaze_locations)
        
        if fixation == dot_loc:
            flag_count += 1 
            r = 0
            
        if flag_count == 4:
            r = 1
            print('correct')
            correct_response_count += 1 # used to track performance
        '''
        if flag_count == 4:
            r = 1
            correct_response_count += 1
            
        value = values[fixation]
        error = r - value
        W += lrate*log_transform(error)*eligibility
        
        #print(fixation)
        if trial % 100 == 0:
            performance = correct_response_count/count
            print('Trial:',trial)
            print('Performance:',performance)
            count = 0
            correct_response_count = 0
    try:   
        performance = correct_response_count/count
        print('Trial:',trial)
        print('Performance:',performance)
        count = 0
        correct_response_count = 0
    except ZeroDivisionError:
        print('Trial:',trial)
        print('Performance:',0)
    

In [46]:
Delayed_Saccade(100000,.1,.5,.9,.1)
# (num trials, learning rate, discount factor(gamma), lambda, temperature)

Trial: 0
Performance: 1.0
Trial: 100
Performance: 0.65
Trial: 200
Performance: 0.79
Trial: 300
Performance: 0.92
Trial: 400
Performance: 0.92
Trial: 500
Performance: 0.87
Trial: 600
Performance: 0.98
Trial: 700
Performance: 0.98
Trial: 800
Performance: 0.96
Trial: 900
Performance: 0.96
Trial: 1000
Performance: 0.9
Trial: 1100
Performance: 0.98
Trial: 1200
Performance: 0.91
Trial: 1300
Performance: 0.98
Trial: 1400
Performance: 0.99
Trial: 1500
Performance: 0.98
Trial: 1600
Performance: 0.95
Trial: 1700
Performance: 0.94
Trial: 1800
Performance: 0.98
Trial: 1900
Performance: 0.96
Trial: 2000
Performance: 0.95
Trial: 2100
Performance: 0.97
Trial: 2200
Performance: 1.0
Trial: 2300
Performance: 0.91
Trial: 2400
Performance: 0.93
Trial: 2500
Performance: 0.94
Trial: 2600
Performance: 0.94
Trial: 2700
Performance: 0.96
Trial: 2800
Performance: 0.98
Trial: 2900
Performance: 0.96
Trial: 3000
Performance: 0.91
Trial: 3100
Performance: 0.97
Trial: 3200
Performance: 0.96
Trial: 3300
Performance: 

/Users/arthurwilliams/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning:

overflow encountered in exp

/Users/arthurwilliams/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning:

invalid value encountered in true_divide



Trial: 4900
Performance: 0.94
Trial: 5000
Performance: 0.86
Trial: 5100
Performance: 0.95
Trial: 5200
Performance: 0.98
Trial: 5300
Performance: 0.89
Trial: 5400
Performance: 0.96
Trial: 5500
Performance: 0.96
Trial: 5600
Performance: 0.98
Trial: 5700
Performance: 0.94
Trial: 5800
Performance: 0.95
Trial: 5900
Performance: 0.94
Trial: 6000
Performance: 0.94
Trial: 6100
Performance: 0.97
Trial: 6200
Performance: 0.96
Trial: 6300
Performance: 0.94
Trial: 6400
Performance: 0.95
Trial: 6500
Performance: 0.89
Trial: 6600
Performance: 0.98
Trial: 6700
Performance: 0.92
Trial: 6800
Performance: 0.89
Trial: 6900
Performance: 0.96
Trial: 7000
Performance: 0.93
Trial: 7100
Performance: 0.94
Trial: 7200
Performance: 0.97
Trial: 7300
Performance: 0.95
Trial: 7400
Performance: 0.91
Trial: 7500
Performance: 0.96
Trial: 7600
Performance: 0.85
Trial: 7700
Performance: 0.98
Trial: 7800
Performance: 0.91
Trial: 7900
Performance: 0.96
Trial: 8000
Performance: 0.96
Trial: 8100
Performance: 0.94
Trial: 820

Trial: 31600
Performance: 0.87
Trial: 31700
Performance: 0.89
Trial: 31800
Performance: 0.89
Trial: 31900
Performance: 0.85
Trial: 32000
Performance: 0.93
Trial: 32100
Performance: 0.9
Trial: 32200
Performance: 0.9
Trial: 32300
Performance: 0.85
Trial: 32400
Performance: 0.94
Trial: 32500
Performance: 0.91
Trial: 32600
Performance: 0.92
Trial: 32700
Performance: 0.91
Trial: 32800
Performance: 0.89
Trial: 32900
Performance: 0.93
Trial: 33000
Performance: 0.86
Trial: 33100
Performance: 0.93
Trial: 33200
Performance: 0.93
Trial: 33300
Performance: 0.9
Trial: 33400
Performance: 0.92
Trial: 33500
Performance: 0.89
Trial: 33600
Performance: 0.93
Trial: 33700
Performance: 0.88
Trial: 33800
Performance: 0.96
Trial: 33900
Performance: 0.87
Trial: 34000
Performance: 0.94
Trial: 34100
Performance: 0.94
Trial: 34200
Performance: 0.85
Trial: 34300
Performance: 0.92
Trial: 34400
Performance: 0.92
Trial: 34500
Performance: 0.95
Trial: 34600
Performance: 0.87
Trial: 34700
Performance: 0.86
Trial: 3480

Trial: 58200
Performance: 0.86
Trial: 58300
Performance: 0.87
Trial: 58400
Performance: 0.89
Trial: 58500
Performance: 0.85
Trial: 58600
Performance: 0.86
Trial: 58700
Performance: 0.91
Trial: 58800
Performance: 0.92
Trial: 58900
Performance: 0.91
Trial: 59000
Performance: 0.83
Trial: 59100
Performance: 0.88
Trial: 59200
Performance: 0.91
Trial: 59300
Performance: 0.91
Trial: 59400
Performance: 0.74
Trial: 59500
Performance: 0.9
Trial: 59600
Performance: 0.86
Trial: 59700
Performance: 0.89
Trial: 59800
Performance: 0.87
Trial: 59900
Performance: 0.93
Trial: 60000
Performance: 0.83
Trial: 60100
Performance: 0.9
Trial: 60200
Performance: 0.8
Trial: 60300
Performance: 0.92
Trial: 60400
Performance: 0.91
Trial: 60500
Performance: 0.86
Trial: 60600
Performance: 0.89
Trial: 60700
Performance: 0.85
Trial: 60800
Performance: 0.86
Trial: 60900
Performance: 0.93
Trial: 61000
Performance: 0.88
Trial: 61100
Performance: 0.89
Trial: 61200
Performance: 0.87
Trial: 61300
Performance: 0.91
Trial: 6140

Trial: 84800
Performance: 0.87
Trial: 84900
Performance: 0.81
Trial: 85000
Performance: 0.85
Trial: 85100
Performance: 0.84
Trial: 85200
Performance: 0.88
Trial: 85300
Performance: 0.9
Trial: 85400
Performance: 0.87
Trial: 85500
Performance: 0.85
Trial: 85600
Performance: 0.87
Trial: 85700
Performance: 0.82
Trial: 85800
Performance: 0.84
Trial: 85900
Performance: 0.89
Trial: 86000
Performance: 0.86
Trial: 86100
Performance: 0.9
Trial: 86200
Performance: 0.84
Trial: 86300
Performance: 0.82
Trial: 86400
Performance: 0.85
Trial: 86500
Performance: 0.91
Trial: 86600
Performance: 0.86
Trial: 86700
Performance: 0.86
Trial: 86800
Performance: 0.87
Trial: 86900
Performance: 0.9
Trial: 87000
Performance: 0.87
Trial: 87100
Performance: 0.81
Trial: 87200
Performance: 0.82
Trial: 87300
Performance: 0.93
Trial: 87400
Performance: 0.86
Trial: 87500
Performance: 0.91
Trial: 87600
Performance: 0.82
Trial: 87700
Performance: 0.89
Trial: 87800
Performance: 0.9
Trial: 87900
Performance: 0.88
Trial: 88000

In [ ]:
l = [0,1,2,3,4]
l[0:1]
#np.argmax([0,1,2,3,4])

In [ ]:
gusget all 1160 LAB
gusrun all 1160 LAB
gus2pdf all 1160 LAB
----------
guscode all 1160 LAB
gusput all 1160 LAB